## 1. 파이썬과 MySQL 활용

### 모듈

1) mysqlclient, PyMySQL

2) pop install mysqlclient, pip install PyMySQL

### 1. mysqlclient 모듈 사용

In [1]:
import MySQLdb

In [2]:
# mysql 서버에 접속

conn = MySQLdb.connect(host='127.0.0.1', user = 'root', password = '1111', 
                       database = 'testdb')
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 000001F4C09C2B38>


In [3]:
config = {'host' : '127.0.0.1', 'user' : 'root', 'password' : '1111', 
          'database' : 'testdb'}

conn = MySQLdb.connect(**config)
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 000001F4C0C291E8>


In [4]:
# SELECT

conn = MySQLdb.connect(**config)

sql = 'SELECT empno, ename, sal, job, deptno FROM emp'

cur = conn.cursor()
cur.execute(sql)

#print(cur)
#print(type(cur))

for data in cur :
    print('%d, %s, %d, %s, %d'%data)        #언패킹
    
#언패킹 두 번째 방법
# for empno, ename, sal, job, deptno in cur :
#   print(empno, ename, sal, job, deptno)

conn.close()

7369, SMITH, 800, CLERK, 20
7499, ALLEN, 1600, SALESMAN, 30
7521, WARD, 1250, SALESMAN, 30
7566, JONES, 2975, MANAGER, 20
7654, MARTIN, 1250, SALESMAN, 30
7698, BLAKE, 2850, MANAGER, 30
7782, CLARK, 2450, MANAGER, 10
7788, SCOTT, 3000, ANALYST, 20
7839, KING, 5000, PRESIDENT, 10
7844, TURNER, 1500, SALESMAN, 30
7876, ADAMS, 1100, CLERK, 20
7900, JAMES, 950, CLERK, 30
7902, FORD, 3000, ANALYST, 20
7934, MILLER, 1300, CLERK, 10


In [5]:
# INSERT

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = '''
INSERT INTO emp(empno, ename, sal, job, mgr, deptno, hiredate)
VALUES(1, 'tom', 700, 'clerk', 7788, 20, now())'''

cur.execute(sql)
conn.commit()

conn.close()

In [6]:
conn = MySQLdb.connect(**config)
cur = conn.cursor()

v_empno = 5
v_ename = 'Johnson'
v_sal = 1500

sql = 'INSERT INTO emp(empno, ename, sal, hiredate) VALUES(' + str(v_empno) + ", '" + v_ename + "', " + str(v_sal) + ', now())'

cur.execute(sql)
conn.commit()

conn.close()

In [7]:
conn = MySQLdb.connect(**config)
cur = conn.cursor()

v_empno = 10
v_ename = 'Jane'
v_sal = 2000

sql = 'INSERT INTO emp(empno, ename, sal, hiredate) VALUES(%s, %s, %s, now())'
sql_data = (str(v_empno), v_ename, str(v_sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()

In [8]:
# DELETE

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = 'DELETE FROM emp WHERE empno IN (1, 5, 10)'

cur.execute(sql)
conn.commit()

conn.close()

### 2. PyMySQL 모듈 사용

In [9]:
import pymysql

In [10]:
conn = pymysql.connect(host='127.0.0.1', user = 'root', password = '1111', 
                       database = 'testdb', port = 3306, charset = 'utf8')

print(conn)
conn.close()

In [11]:
config = {'host' : '127.0.0.1', 'user' : 'root', 'password' : '1111', 
          'database' : 'testdb', 'port' : 3306, 'charset' : 'utf8'}

conn = pymysql.connect(**config)
print(conn)
conn.close()

In [18]:
conn = pymysql.connect(**config)
cur = conn.cursor()

sql = '''CREATE TABLE pythonTable(
id    int    unsigned not null auto_increment,
name    varchar(20)    not null,
regdate    datetime,
primary key(id));'''

cur.execute(sql)
conn.commit()

conn.close()

In [13]:
conn = pymysql.connect(**config)
cur = conn.cursor()

sql = '''
    CREATE TABLE IF NOT EXISTS pythonTable(
            id    int    unsigned not null auto_increment,
            name    varchar(20)    not null,
            regdate    datetime,
            primary key(id));'''

cur.execute(sql)
cur.execute('show tables')

for i in cur :
    print(i)

conn.commit()

conn.close()

('bonus',)
('dept',)
('emp',)
('member',)
('post',)
('pythontable',)
('salgrade',)
('tbla',)
('tblb',)
('tblbook',)
('tblc',)
('tbldept',)
('tblemp',)
('tblindex',)
('tbltest',)
('vw_emp1',)


C:\Users\acorn\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1050, "Table 'pythontable' already exists")
  result = self._query(query)


In [14]:
conn = pymysql.connect(**config)
try :
    cur = conn.cursor()
    
    sql = '''
    INSERT INTO pythontable(name, regdate)
    VALUES('tom', now());'''
    
    for num in range(10) :
        cur.execute(sql)
    
    conn.commit()
finally :
    conn.close()

In [31]:
# csv 파일을 DB에 저장하기

'''
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
'''

import csv
from datetime import datetime

conn = pymysql.connect(**config)

try :
    cur = conn.cursor()
    
    file = csv.reader(open('C:/Users/acorn/hyeonji/database/input.csv'))
    #print(file)
    
    header = next(file)
    
    for row in file :
        data = []
        for col in range(len(header)) :
            if col == 4:
                tmp = datetime.strptime(row[col], '%Y-%m-%d') 
                # 문자열을 날짜로 변경
                data.append(tmp)
            else :
                data.append(row[col])
                
        cur.execute('INSERT INTO suppliers VALUES(%s, %s, %s, %s, %s)', data)
     
    conn.commit()
    
finally :
    conn.close()

In [36]:
# csv 파일을 DB에 저장하기2

import csv
from datetime import datetime

conn = pymysql.connect(**config)

try :
    cur = conn.cursor()
    
    sql = 'delete from suppliers'
    cur.execute(sql)
    conn.commit()
    
    file = csv.reader(open('C:/Users/acorn/hyeonji/database/input.csv'))
    #print(file)
    
    header = next(file)
    
    for row in file :
        cur.execute('INSERT INTO suppliers VALUES(%s, %s, %s, %s, %s)', row)
     
    conn.commit()
    
finally :
    conn.close()

In [42]:
# DB에서 가져온 데이터를 csv로 저장하기

conn = pymysql.connect(**config)

try :
    cur = conn.cursor()
    
    cur.execute('SELECT * FROM suppliers WHERE cost>500000')
    #for i in cur : print(i)
    
    file = csv.writer(open('C:/Users/acorn/hyeonji/database/output.csv', 'w'), delimiter=',')
    
    header = ['Supplier Name', 'Invoice Name', 'Part Number', 'Cost', 'Purchase Date']
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
    
finally :
    conn.close()

## 2. 파이썬과 SQLite

1. 로컬 DB로 사용할 수 있는 제품들
    * Access
    * SQLite
    * Microsoft SQL Server Compact Edition
    * Firebird Embeded Server
    * H2
    * HSQL
    * Apache Derby
    ...

2. 특징
    * 장점
        * 파일 기반의 DBMS, 저메모리, 빠른 처리 속도
        * 오픈 소스
        * 별도의 DB 서버가 없어도 쉽고 편리하게 사용할 수 있는 Embeded SQL DB 엔진
        * 안드로이드, 아이폰 등의 스마트폰에 기본 내장된 DB
        * 표준 SQL 지원
        * 파이썬에 내장
    * 단점
        * 컬럼을 삭제하거나 변경하는 것 등이 제한
        * Left Outer Join만 지원
        * 읽기 전용 뷰만 제공
        
3. 공식 사이트 : http://sqlite.org

4. 클라이언트 툴
    * http://www.sqliteexpert.com

In [47]:
# sqliteexpert를 이용해서 db 생성 : mysqlite.db

'''
CREATE TABLE tblmember(
    id    int,
    name    varchar(10),
    regdate    datetime);
'''

'''
INSERT INTO tblmember VALUES(1, '홍길동', '2020-5-5');
INSERT INTO tblmember VALUES(2, '임꺽정', date('now'));
'''

import sqlite3

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.31.1


In [52]:
conn = sqlite3.connect('C:/Users/acorn/hyeonji/database/mysqlite.db')
print(conn)
cur = conn.cursor()
print(cur)
conn.close()

In [58]:
# SELECT

conn = sqlite3.connect('C:/Users/acorn/hyeonji/database/mysqlite.db')
cur = conn.cursor()

cur.execute('SELECT * FROM tblmember')

for row in cur :
    print(row)
    
conn.close()

(1, '홍길동', '2020-5-5')
(2, '임꺽정', '2020-05-28')


In [63]:
# INSERT

conn = sqlite3.connect('C:/Users/acorn/hyeonji/database/mysqlite.db')
cur = conn.cursor()

cur.execute("INSERT INTO tblmember VALUES(3, '홍경래', date('now'))")

conn.commit()
conn.close()

In [66]:
# csv 파일을 읽어서 DB에 저장

'''
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);
'''

import csv

file = csv.reader(open('C:/Users/acorn/hyeonji/database/input.csv', 'r'), delimiter=',')
header = next(file)


conn = sqlite3.connect('C:/Users/acorn/hyeonji/database/mysqlite.db')
cur = conn.cursor()


for row in file :
    cur.execute('INSERT INTO suppliers VALUES(?, ?, ?, ?, ?)', row)
    
    
conn.commit()   
conn.close()

In [67]:
"""
1) mysql의 emp테이블을 emp.csv로 저장

2) emp테이블을 이용해서 아래와 같은 프로그램

예시)
        1. mysql
        2. sqlite
        연결할 db의 종류 :
        
        연결할 주소와 계정을 입력: (mysql일 경우)
        주소 : xxx.xxx.xxx.xxx
        아이디 : xxx
        패스워드 : xxx
        
        메뉴 선택
        1. 조회
        2. 입력
        3. 수정
        4. 삭제
        메뉴 : 
    
"""

'\n1) mysql의 emp테이블을 emp.csv로 저장\n\n2) emp테이블을 이용해서 아래와 같은 프로그램\n\n예시)\n        1. mysql\n        2. sqlite\n        연결할 db의 종류 :\n        \n        연결할 주소와 계정을 입력: (mysql일 경우)\n        주소 : xxx.xxx.xxx.xxx\n        아이디 : xxx\n        패스워드 : xxx\n        \n        메뉴 선택\n        1. 조회\n        2. 입력\n        3. 수정\n        4. 삭제\n        메뉴 : \n    \n'

In [70]:
# 메모리 DB

conn = sqlite3.connect(":memory:")
cur = conn.cursor()
#print(cur)

cur.execute("CREATE TABLE phonebook(name text, phonenum varchar(20));")
cur.execute("INSERT INTO phonebook VALUES(?, ?);", ("Tom", "010-111-1111"))

In [71]:
cur.execute("select * from phonebook")

for i in cur:
    print(i)

('Tom', '010-111-1111')


In [78]:
# 입력 데이터 인자를 변수명 처리

sql = "INSERT INTO phonebook VALUES(:vName, :vPhone)"
cur.execute(sql, {"vPhone" : "010-222-2222", "vName" : "Johnson"})

In [79]:
# 여러 개의 데이터를 한 번에 처리 : executemany

datalist = (('jane', '010-333-3333'), ('paul', '010-444-4444'), ('chris', '010-555-5555'))

cur.executemany("INSERT INTO phonebook VALUES(?, ?);", datalist)

In [80]:
# 입력 데이터가 여러 개일 때 : fetchmany

cur.execute("select * from phonebook")

for i in cur.fetchmany(3) :
    print(i)

('Tom', '010-111-1111')
('Johnson', '010-222-2222')
('jane', '010-333-3333')


In [82]:
# 작업한 내용을 스크립트로 저장

f = open('C:/Users/acorn/hyeonji/database/dump.sql', 'w')

for line in conn.iterdump() :
    #print(line)
    f.write("{}\n".format(line))

f.close()
conn.close()

In [84]:
# 스크립트 불러오기

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

f = open('C:/Users/acorn/hyeonji/database/dump.sql')
script = f.read()
#print(script)

cur.executescript(script)

cur.execute("select * from phonebook")

for i in cur.fetchmany(3) :
    print(i)
    
f.close()
conn.close()

('Tom', '010-111-1111')
('Johnson', '010-222-2222')
('jane', '010-333-3333')


## 3. MongoDB와 연동

pip install pymongo

In [3]:
# 방법 1
import pymongo

conn = pymongo.MongoClient("127.0.0.1", 27017)
print(conn)
conn.close()

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


In [2]:
# 방법 2
from pymongo import MongoClient

conn = MongoClient("mongodb://127.0.0.1:27017")
print(conn)
conn.close()

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


In [6]:
# DB 연결

conn = MongoClient("mongodb://127.0.0.1:27017")

#db = conn.testdb
#db = conn.testdb2    -> 새로운 db 생성
db = conn["testdb"]

print(db)

conn.close()

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'testdb')


In [7]:
# Collection과 연결

conn = MongoClient("mongodb://127.0.0.1:27017")

db = conn["testdb"]
#coll = db.articles
#coll = db["articles"]
coll = db.get_collection("articles")

print(coll)

conn.close()

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'testdb'), 'articles')


In [10]:
# 데이터 한 개 입력
## coll.count() : coll에 몇 개의 document가 있는지  확인
## coll.insert_one({"키" : "값"})

conn = MongoClient("mongodb://127.0.0.1:27017")

db = conn["testdb"]
coll = db.people

print(coll.count())

#data = {"author" : "Mike", "text" : "my first blog post!", 
#        "tag" : ["mongodb", "python"]}
#coll.insert_one(data)

data = {"author" : "Mike", "text" : "my second blog post!", 
        "tag" : ["mongodb", "python"]}
post_id = coll.insert_one(data).inserted_id
print(post_id)

conn.close()

6
5ed0bdcb29e37669e377bb6c


C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  # Remove the CWD from sys.path while we load stuff.


In [11]:
# 데이터 여러 개 입력

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn["testdb"]
coll = db.people

print(coll.count())

'''
data = []
for i in range(10) :
    temp = {"number" : i}
    data.append(temp)
        
coll.insert_many(data)
'''

coll.insert_many([{"number" : i} for i in range(10)])

print(coll.count())

conn.close()

7
17


C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  import sys
C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere


In [9]:
# 조회

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn["testdb"]
coll = db.people

#for i in coll.find():
#    print(i)


# age가 20인 document 조회
for i in coll.find({"age" : 20}) :
    print(i)
    print('-'*85)

# age가 20 이상인 document 조회
for i in coll.find({"age" : {"$gte" : 20}}) :
    print(i)

print(coll.find_one({"author" : "Mike"}))
    
conn.close()

{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf1'), 'name': 'Abet', 'age': 20.0, 'score': 10.0}
-------------------------------------------------------------------------------------
{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf2'), 'name': 'Betty', 'age': 20.0, 'score': 10.0}
-------------------------------------------------------------------------------------
{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf1'), 'name': 'Abet', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf2'), 'name': 'Betty', 'age': 20.0, 'score': 10.0}
{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf3'), 'name': 'Charlie', 'age': 23.0, 'skills': ['mongodb', 'python']}
{'_id': ObjectId('5ed0a24d7277e7fa4cc06bf4'), 'name': 'David', 'age': 23.0, 'score': 20.0}


In [ ]:
# 데이터 수정

conn = MongoClient("mongodb://127.0.0.1:27017")
db = conn["testdb"]
coll = db.people

coll.update_one({"author" : "Mike"}, {"$set" :{"text" : "my third blog post!"}})

print(coll.find_one({"author" : "Mike"}))

conn.close()